In [1]:
# starting a spark session
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('lreg').getOrCreate()

In [3]:
%fs ls /FileStore/tables/

path,name,size
dbfs:/FileStore/tables/ContainsNull.csv,ContainsNull.csv,61
dbfs:/FileStore/tables/Spark_Essentials-5d27c.dbc,Spark_Essentials-5d27c.dbc,1414841
dbfs:/FileStore/tables/appl_stock.csv,appl_stock.csv,143130
dbfs:/FileStore/tables/cogsley_clients.csv,cogsley_clients.csv,384219
dbfs:/FileStore/tables/cogsley_sales.csv,cogsley_sales.csv,2176442
dbfs:/FileStore/tables/people.json,people.json,73
dbfs:/FileStore/tables/sales_info.csv,sales_info.csv,196
dbfs:/FileStore/tables/sample_linear_regression_data.txt,sample_linear_regression_data.txt,119069
dbfs:/FileStore/tables/state_info.csv,state_info.csv,2778
dbfs:/FileStore/tables/walmart_stock.csv,walmart_stock.csv,90266


In [4]:
path = '/FileStore/tables/sample_linear_regression_data.txt'

In [5]:
training_data = spark.read.format('libsvm').load(path)

In [6]:
training_data.show(5)

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows

In [7]:
from pyspark.ml.regression import LinearRegression

In [8]:
lr = LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction')

In [9]:
lr_model = lr.fit(training_data)

In [10]:
lr_model.coefficients

Out[ 11 ]: DenseVector([0.0073, 0.8314, -0.8095, 2.4412, 0.5192, 1.1535, -0.2989, -0.5129, -0.6197, 0.6956])

In [11]:
lr_model.intercept

Out[ 12 ]: 0.14228558260358093

In [12]:
training_summary = lr_model.summary

In [13]:
training_summary.pValues

Out[ 23 ]: 
[0.9927505031240562,
 0.30967074330990396,
 0.3178269194409711,
 0.003972477331573909,
 0.5201486327242175,
 0.16213017210149872,
 0.7102819001865635,
 0.5266812209137877,
 0.46256007153356316,
 0.37825808848978526,
 0.7592692146070568]

In [14]:
# starting again from the begining with train test split

In [15]:
df = spark.read.format('libsvm').load(path)

In [16]:
#spliting the data by tuple unpacking
train_data,test_data = df.randomSplit([0.7,0.3])

In [17]:
train_data.describe().show()

+-------+--------------------+
summary| label|
+-------+--------------------+
 count| 341|
 mean|-0.44638728003370653|
 stddev| 10.140763683826565|
 min| -26.736207182601724|
 max| 27.78383192005107|
+-------+--------------------+

In [18]:
model = lr.fit(train_data)

In [19]:
model_summary = model.summary

In [20]:
model_summary.pValues

Out[ 33 ]: 
[0.5364705480946161,
 0.6642759989707918,
 0.6834783585071018,
 0.018709393750457437,
 0.36718819133473235,
 0.10587056545819284,
 0.37214423332049473,
 0.874313139767052,
 0.35148944782892455,
 0.23440970741413203,
 0.25068598246983154]

In [21]:
test_result = model.evaluate(test_data)

In [22]:
test_result.residuals.show()

+-------------------+
 residuals|
+-------------------+
-27.345061477647235|
 -25.08277880465579|
-25.986391195416722|
 -22.54224205459704|
-17.290349393280913|
-14.271849043061861|
-15.837558812016526|
 -16.61795769552071|
 -16.67082709916258|
 -17.0421588581746|
-12.456354572259379|
-11.467953344288965|
-11.027338757353752|
 -11.61457205423446|
 -14.02542414290868|
-13.169362532416214|
 -8.779456206090288|
 -9.109494627063203|
 -9.80192708450865|
 -8.365090226441243|
+-------------------+
only showing top 20 rows

In [23]:
test_result.meanSquaredError

Out[ 40 ]: 117.29443644030948

In [24]:
model_summary.meanSquaredError

Out[ 41 ]: 98.73719370522288

In [25]:
# mimicing the real world model evaluation, we are using test data but only with x
unlabeled_data = test_data.select('features')

In [26]:
pred = model.transform(unlabeled_data)

In [27]:
pred.show()

+--------------------+-------------------+
 features| prediction|
+--------------------+-------------------+
(10,[0,1,2,3,4,5,...|-1.2264173920961903|
(10,[0,1,2,3,4,5,...|-2.9632392331208446|
(10,[0,1,2,3,4,5,...|-0.8190922330663496|
(10,[0,1,2,3,4,5,...|-0.9686420357259304|
(10,[0,1,2,3,4,5,...| 1.0289191178281807|
(10,[0,1,2,3,4,5,...|-1.8138099979596292|
(10,[0,1,2,3,4,5,...|-0.1139537537780464|
(10,[0,1,2,3,4,5,...| 0.7559483679501479|
(10,[0,1,2,3,4,5,...| 0.9473114861140094|
(10,[0,1,2,3,4,5,...| 1.7073913782522583|
(10,[0,1,2,3,4,5,...|-1.4107333228993895|
(10,[0,1,2,3,4,5,...|-1.6853822620765664|
(10,[0,1,2,3,4,5,...|-1.4403176236791082|
(10,[0,1,2,3,4,5,...| -0.583524510426953|
(10,[0,1,2,3,4,5,...| 2.14725664294184|
(10,[0,1,2,3,4,5,...| 1.7375601668757543|
(10,[0,1,2,3,4,5,...| -1.798242098152934|
(10,[0,1,2,3,4,5,...|-1.3796624963096957|
(10,[0,1,2,3,4,5,...|-0.4937388515212694|
(10,[0,1,2,3,4,5,...|-1.9235670259474644|
+--------------------+-------------------+
only showing top 20 rows